# 1.数据预处理

In [8]:
# 1.数据初探
# 1.1首先导入要使用的科学计算包numpy，pandas，可视化matplotlib，seaborn,以及机器学习包sklearn。
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl

import matplotlib.pyplot as plt
from IPython.display import display

plt.style.use("fivethirtyeight")
sns.set_style({'font.sans-serif': ['simhei', 'Arial']})
# % matplotlib inline

In [ ]:
# 1.2然后导入数据，并进行初步的观察，这些观察包括了解数据特征的缺失值，异常值，以及大概的描述性统计。
# 导入链家房屋数据
data = pd.read_csv("./data.csv")
display(data.head(n=2))

In [ ]:
# 检查缺失值情况
data.info

In [ ]:
# 1.3初步观察到一共有11个特征变量，Price 在这里是我们的目标变量，然后我们继续深入观察一下。
# 检查缺失值情况
data.describe

In [ ]:
# 添加新特征房屋均价
df = data.copy()
df['PerPrice'] = data['Price'] / data['Size']
df[:2]

In [ ]:
# 重新摆放列位置
columns = ['Region', 'District', 'Garden', 'Layout', 'Floor', 'Year', 'Size', 'Elevator', 'Direction', 'Renovation',
           'PerPrice', 'Price']

df = pd.DataFrame(df, columns=columns)
# 重新审视数据集
display(df.head(n=2))


In [ ]:
# 2 数据可视化分析
# 2.1 Region特征分析

# 对于区域特征，我们可以分析不同区域房价和数量的对比。
# 对房屋区域分组对比房屋数量和每平米房价
# 使用了pandas的网络透视功能 groupby 分组排序。
# 区域特征可视化直接采用 seaborn 完成，颜色使用调色板 palette 参数，颜色渐变，越浅说明越少，反之越多
df_house_count = df.groupby('Region')['Price'].count().sort_values(ascending=False).to_frame().reset_index()
df_house_mean = df.groupby('Region')['PerPrice'].mean().sort_values(ascending=False).to_frame().reset_index()

f, [ax1, ax2, ax3] = plt.subplots(3, 1, figsize=(20, 15))
sns.barplot(x='Region', y='PerPrice', palette="Blues_d", data=df_house_mean, ax=ax1)
ax1.set_title('北京各大区房屋每平米单价对比', fontsize=12)
ax1.set_xlabel('区域', fontsize=12)
ax1.set_ylabel('每平米单价', fontsize=12)

sns.barplot(x='Region', y='Price', palette="Greens_d", data=df_house_count, ax=ax2)
ax2.set_title('北京各大区房屋数量对比', fontsize=12)
ax2.set_xlabel('区域', fontsize=12)
ax2.set_ylabel('数量', fontsize=12)

sns.boxplot(x='Region', y='Price', data=df, ax=ax3)
ax3.set_title('北京各大区房屋房屋总价', fontsize=12)
ax3.set_xlabel('区域', fontsize=12)
ax3.set_ylabel('房屋总价', fontsize=12)

# plt.show()
plt.savefig("Region.png")

In [ ]:
# 2.2 Size特征分析
f, [ax1, ax2] = plt.subplots(1, 2, figsize=(15, 10))

# 房屋面积的分布情况
sns.distplot(df['Size'], bins=20, ax=ax1, color='r')
sns.kdeplot(df['Size'], shade=True, ax=ax1)
ax1.set_title('房屋面积的分布情况', fontsize=12)
ax1.set_xlabel('面积/平米', fontsize=12)
ax1.set_ylabel('所占比例', fontsize=12)

# 建房时间和出售价格的关系
sns.regplot(x='Size', y='Price', data=df, ax=ax2)
ax2.set_title('建房时间和出售价格的关系', fontsize=12)
ax2.set_xlabel('面积/平米', fontsize=12)
ax2.set_ylabel('价格', fontsize=12)
# plt.show()
plt.savefig('Size.png')

In [ ]:
print("房屋面积小于10平米：")
df.loc[df['Size'] < 10]

In [ ]:
print("房屋面积大于1000平米：")
df.loc[df['Size'] > 1000]

In [ ]:
# 2.3 Layout特征分析
f, ax1 = plt.subplots(figsize=(20, 20))
sns.countplot(y='Layout', data=df, ax=ax1)
ax1.set_title('房屋户型与数量关系', fontsize=12)
ax1.set_xlabel('数量', fontsize=12)
ax1.set_ylabel('户型', fontsize=12)
# plt.show()
plt.savefig('Layout.png')

In [ ]:
# 2.4Renovation特征分析
print(df['Renovation'].value_counts())
# 去掉数据中装修特征“南北”
df['Renovation'] = df.loc[(df['Renovation'] != '南北'), 'Renovation']
# 画幅设置
f, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=(20, 10))
sns.countplot(df['Renovation'], ax=ax1)
ax1.set_title('装修特征与数量关系', fontsize=12)
ax1.set_xlabel('装修特征', fontsize=12)
ax1.set_ylabel('数量', fontsize=12)

sns.barplot(x='Renovation', y='Price', data=df, ax=ax2)
ax2.set_title('装修特征与价格关系', fontsize=12)
ax2.set_xlabel('装修特征', fontsize=12)
ax2.set_ylabel('价格', fontsize=12)

sns.boxplot(x='Renovation', y='Price', data=df, ax=ax3)
ax3.set_title('装修特征与数量关系箱型图', fontsize=12)
ax3.set_xlabel('装修特征', fontsize=12)
ax3.set_ylabel('数量', fontsize=12)
# plt.show()
plt.savefig('Renovation.png')

In [ ]:
# 2.5Elevator特征分析
# 初探数据时，Elevator有大量的缺失值
misn = len(df.loc[(df['Elevator'].isnull()), 'Elevator'])
print("电梯的缺失值数量：", misn)

# 由于存在个别类型错误，如简装和精装，特征值错位，故需要移除
df['Elevator'] = df.loc[(df['Elevator'] == '有电梯') | (df['Elevator'] == '无电梯'), 'Elevator']

# 填补Elevator缺失值  随机的
df.loc[(df['Floor'] > 6) & (df['Elevator'].isnull()), 'Elevator'] = '有电梯'
df.loc[(df['Floor'] <= 6) & (df['Elevator'].isnull()), 'Elevator'] = '无电梯'

f, [ax1, ax2] = plt.subplots(1, 2, figsize=(20, 10))
sns.countplot(df['Elevator'], ax=ax1)
ax1.set_title('有无电梯数量对比', fontsize=15)
ax1.set_xlabel('是否有电梯')
ax1.set_ylabel('数量')
sns.barplot(x='Elevator', y='Price', data=df, ax=ax2)
ax2.set_title('有无电梯房价对比', fontsize=15)
ax2.set_xlabel('是否有电梯')
ax2.set_ylabel('总价')
# plt.show()
plt.savefig("Elevator.png")

In [ ]:
#2.6 Year特征分析
# grid = sns.FacetGrid(df, row='Elevator', col='Renovation', palette='seismic',size=4)
grid = sns.FacetGrid(df, row='Elevator', col='Renovation', palette='seismic',height=4)
grid.map(plt.scatter, 'Year', 'Price')
grid.add_legend()
grid.savefig("Year.png")